In [ ]:
from link import Link
from node import Node
from path import Path
from od import OD

import time
import sys
import traceback
import utils
import math


FRANK_WOLFE_STEPSIZE_PRECISION = 1e-7


class BadNetworkOperationException(Exception):
    """
    You can raise this exception if you try a network action which is invalid
    (e.g., trying to find a topological order on a network with cycles.)
    """
    pass


class Network:

    def __init__(self, networkFile="", demandFile=""):
        """
        Class initializer; if both a network file and demand file are specified,
        will read these files to fill the network data structure.
        """
        self.time = time
        self.numNodes = 0
        self.numLinks = 0
        self.numZones = 0
        self.firstThroughNode = 0

        self.node = dict()
        self.link = dict()
        self.ODpair = dict()
        self.path = dict()
        self.origin_set = []

        if len(networkFile) > 0 and len(demandFile) > 0:
            self.readFromFiles(networkFile, demandFile)

    def writeUEresults(self):
        # IVT, WT, WK, TR
        BFW_outFile = open("无敌1_GP_link_results.dat", "w")
        tmpOut = "Tail_node\tHead_node\tTravel_time\tFlow"
        BFW_outFile.write(tmpOut+"\n")
        for ij in self.link:
            tmpOut = str(self.link[ij].tail) + "\t" + str(self.link[ij].head) + \
                "\t" + str(self.link[ij].flow) + "\t" + str(self.link[ij].cost)
            BFW_outFile.write(tmpOut + "\n")
        BFW_outFile.close()
        # IVT, WT, WK, TR
        BFW_outFile = open("无敌GP_OD_results.dat", "w")
        tmpOut = "Origin\tDestination\tDemand\tLeastCost"
        BFW_outFile.write(tmpOut+"\n")
        for OD in self.ODpair:
            tmpOut = str(self.ODpair[OD].origin) + "\t" + str(self.ODpair[OD].destination)\
                + "\t" + str(self.ODpair[OD].demand) + "\t" + \
                str(self.ODpair[OD].fastest_path_time)
            BFW_outFile.write(tmpOut + "\n")
        BFW_outFile.close()

    def relativeGap(self):
        """
        This method should calculate the relative gap (as defined in the course text)
        based on the current link flows, and return this value.

        To do this, you will need to calculate both the total system travel time, and
        the shortest path travel time (you will find it useful to call some of the
        methods implemented in earlier assignments).
        """

        total_system_travel_time = 0
        for ij in self.link:
            travel_time = self.link[ij].flow * self.link[ij].cost
            total_system_travel_time += travel_time
        total_system_travel_time_OD = 0

        for OD in self.ODpair:
            travel_time_OD = self.ODpair[OD].fastest_path_time * self.ODpair[OD].demand
            total_system_travel_time_OD += travel_time_OD
            

        return abs(1 - (total_system_travel_time_OD / total_system_travel_time))

    def averageExcessCost(self):
        """
        This method should calculate the average excess cost
        based on the current link flows, and return this value.

        To do this, you will need to calculate both the total system travel time, and
        the shortest path travel time (you will find it useful to call some of the
        methods implemented in earlier assignments).
        """
        total_system_travel_time = 0
        for ij in self.link:
            travel_time = self.link[ij].flow * self.link[ij].cost
            total_system_travel_time += travel_time

        OD_demand = 0
        total_system_travel_time_OD = 0

        for origin in self.origin_set:
            (backlink, cost) = self.shortestPath(origin)
            for OD in [OD for OD in self.ODpair if self.ODpair[OD].origin == origin]:
                destination = self.ODpair[OD].destination
                self.ODpair[OD].backlink = backlink
                self.ODpair[OD].cost = cost
                self.ODpair[OD].fastest_path_time = cost[destination]-cost[origin]
                travel_time_OD = self.ODpair[OD].fastest_path_time * \
                    self.ODpair[OD].demand
                total_system_travel_time_OD += travel_time_OD
                OD_demand += self.ODpair[OD].demand

        return (total_system_travel_time - total_system_travel_time_OD) / OD_demand
    
    def beckmannFunction(self):
        """
        This method evaluates the Beckmann function at the current link flows.

        """
        beckmann = 0
        for ij in self.link:
            beckmann += self.link[ij].calculateBeckmannComponent()
        return beckmann

    def shortestPath(self, origin):
        """
        This method finds the shortest path in a network which may or may not have
        cycles; thus you cannot assume that a topological order exists.

        The implementation in the text uses a vector of backnode labels.  In this
        assignment, you should use back-LINK labels instead.  The idea is exactly
        the same, except you are storing the ID of the last *link* in a shortest
        path to each node.

        Use the 'cost' attribute of the Links to calculate travel times.  These values
        are given -- do not try to recalculate them based on flows, BPR functions, etc.

        The backlink and cost labels are both stored in dict's, whose keys are
        node IDs.

        *** BE SURE YOUR IMPLEMENTATION RESPECTS THE FIRST THROUGH NODE!
        *** Travelers should not be able to use "centroid connectors" as shortcuts,
        *** and the shortest path tree should reflect this.

        You should use the macro utils.NO_PATH_EXISTS to initialize backlink labels,
        and utils.INFINITY to initialize cost labels.
        """
        backlink = dict()
        cost = dict()

        for i in self.node:
            backlink[i] = utils.NO_PATH_EXISTS
            cost[i] = utils.INFINITY
        cost[origin] = 0

        scanList = [self.link[ij].head for ij in self.node[origin].forwardStar]

        while len(scanList) > 0:
            i = scanList[0]
            scanList.remove(i)
            labelChanged = False
            for hi in self.node[i].reverseStar:
                h = self.link[hi].tail
                if h < self.firstThroughNode and h != origin:
                    continue
                tempCost = cost[h] + self.link[hi].cost
                if tempCost < cost[i]:
                    cost[i] = tempCost
                    backlink[i] = hi
                    labelChanged = True
            if labelChanged == True:
                scanList.extend([self.link[ij].head for ij in self.node[i].forwardStar
                                 if self.link[ij].head not in scanList])

        return (backlink, cost)

    

    def loadPaths(self):
        """
        This method should take given values of path flows (stored in the
        self.path[].flow attributes), and do the following:
           1. Set link flows to correspond to these values (self.link[].flow)
           2. Set link costs based on new flows (self.link[].cost), see link.py
           3. Set path costs based on new link costs (self.path[].cost), see path.py
        """
        for ij in self.link:
            self.link[ij].flow = 0
        for p in self.path:
            for ij in self.path[p].links:
                self.link[ij].flow += self.path[p].flow
        for ij in self.link:
            self.link[ij].updateCost()
        for p in self.path:
            self.path[p].updateCost()

    def improvedGradientProjection(self, maxIterations=10, targetGap=1e-4, gapFunction=relativeGap):
        '''
        relativeGap, averageExcessCost
        '''
        
        t1 = self.time.time()
        BFW_outFile = open("贪婪五GP_convergence_log", "w") 
        tmpOut = 'iteration\tgap'
        BFW_outFile.write(tmpOut+"\n")
        
        
        def bisectopt(function, x_lower, x_upper, tolerance):
                
                    if function(x_upper)>=0:
                        return x_upper
                    else:
                        while (x_upper-x_lower)>tolerance:
                            x = (x_lower + x_upper)/2.0
                            if function(x) == 0:
                                return x
                            elif function(x) < 0:
                                x_upper = x
                            else:
                                x_lower = x
                        return (x_lower + x_upper)/2.0
                

        # setting up OD_Path_Set for evey OD
        for OD in self.ODpair:
            self.ODpair[OD].Path_Set = list()
            
#         list1=dict()
#         for origin in self.origin_set:
#                 a = 0
#                 for OD in [OD for OD in self.ODpair if self.ODpair[OD].origin == origin]:
#                     a += self.ODpair[OD].demand
#                 list1[origin] = a
#         sortedList1 = list(list1.items())
# #         ,reverse =True
#         sortedList1.sort(key=lambda item : item[1],reverse =True)
#         self.origin_set = [i[0] for i in sortedList1]

        for origin in self.origin_set:
            (backlink, cost) = self.shortestPath(origin)
            fresh_links = list()
            for OD in [OD for OD in self.ODpair if self.ODpair[OD].origin == origin]:
                destination = self.ODpair[OD].destination
                self.ODpair[OD].fastest_path_time = cost[destination]
                links = list()
                while destination != origin:
                    links.insert(0, backlink[destination])
                    destination = self.link[backlink[destination]].tail
                self.path[tuple(links)] = Path(tuple(links), self, self.ODpair[OD].demand)
                self.ODpair[OD].Path_Set.append(tuple(links))
                 
                for ij in links:
                    self.link[ij].flow += self.path[tuple(links)].flow
                fresh_links.extend(links)
            for ij in set(fresh_links):
                self.link[ij].updateCost()
        gap = gapFunction()
        iteration = 1    
        print("Iteration %d: gap %f" % (iteration, gap))
        while iteration < maxIterations:
         
            iteration += 1
            for origin in self.origin_set:
                (backlink, cost) = self.shortestPath(origin)
                link_flow_fresh_set = []
                for OD in [OD for OD in self.ODpair if self.ODpair[OD].origin == origin]:
                    destination = self.ODpair[OD].destination
                    self.ODpair[OD].fastest_path_time = cost[destination]
                    links = list()
                    while destination != origin:
                        links.insert(0, backlink[destination])
                        destination = self.link[backlink[destination]].tail
                    if tuple(links) not in self.ODpair[OD].Path_Set:
                        self.path[tuple(links)] = Path(tuple(links), self)
                        self.ODpair[OD].Path_Set.append(tuple(links))
                   
                    
                    a = 0
                    b = math.exp(100)
                    n = 0
                    for p in self.ODpair[OD].Path_Set:
                        n += 1
                        path_cost = 0
                        for ij in self.path[p].links:
                            vcRatio = (self.link[ij].flow)/self.link[ij].capacity
                            path_cost += self.link[ij].freeFlowTime * (1 + self.link[ij].alpha\
                                        * pow(vcRatio, self.link[ij].beta)) +self.link[ij].toll * self.tollFactor + self.link[ij].length * self.distanceFactor
                        if path_cost>=a:
                            a = path_cost
                            path_long = p
                            path_l_cost = path_cost
                        if path_cost<=b:
                            b = path_cost
                            path_short = p
                            path_s_cost = path_cost
#                         self.path[p].updateCost()
                    if (n % 2) == 0:
                        n = n/2
                    else:
                        n = (n-1)/2
                    
                    while n>0:
                            n -= 1
#                             print(OD)
#                             print(path_l_cost)
#                             print(path_s_cost)
                            p = path_long
                            links = path_short
                            link_s = list(set(self.path[p].links)-set(self.path[tuple(links)].links))
                            link_a = list(set(self.path[tuple(links)].links) - set(self.path[p].links))
                            def st(step):
                                path_cost = 0
                                path_s_cost = 0
                                for ij in link_s:
                                    flow = self.link[ij].flow - self.path[p].flow
                                    vcRatio = (self.path[p].flow - step + flow)/self.link[ij].capacity
                                    path_cost += self.link[ij].freeFlowTime * (1 + self.link[ij].alpha\
                                        * pow(vcRatio, self.link[ij].beta)) +self.link[ij].toll * self.tollFactor + self.link[ij].length * self.distanceFactor
                                for ij in link_a:
                                    flow = self.link[ij].flow - self.path[tuple(links)].flow
                                    vcRatio = (self.path[tuple(links)].flow + step + flow)/self.link[ij].capacity
                                    path_s_cost += self.link[ij].freeFlowTime * (1 + self.link[ij].alpha\
                                        * pow(vcRatio, self.link[ij].beta))+self.link[ij].toll * self.tollFactor + self.link[ij].length * self.distanceFactor
                                function =  path_cost - path_s_cost
                                return function
                            shift_flow = bisectopt(st, 0, self.path[p].flow, tolerance = 1e-10)
                            self.path[tuple(links)].flow += shift_flow
                            self.path[p].flow -= shift_flow
                            for ij in link_s:
                                self.link[ij].flow -= shift_flow
                            for ij in link_a:
                                self.link[ij].flow += shift_flow
                                
                            if self.path[p].flow == 0:
                                try:
                                    self.ODpair[OD].Path_Set.remove(p) 
                                except:
                                        pass
                               
                            link_flow_fresh_set.extend(link_s + link_a)
                            
                            a = 0
                            b = math.exp(100)
                            for p in self.ODpair[OD].Path_Set:
                                path_cost = 0
                                for ij in self.path[p].links:
                                    vcRatio = (self.link[ij].flow)/self.link[ij].capacity
                                    path_cost += self.link[ij].freeFlowTime * (1 + self.link[ij].alpha\
                                                * pow(vcRatio, self.link[ij].beta)) +self.link[ij].toll * self.tollFactor + self.link[ij].length * self.distanceFactor
                                if path_cost>=a:
                                    a = path_cost
                                    path_long = p
                                    path_l_cost = path_cost
                                if path_cost<=b:
                                    b = path_cost
                                    path_short = p
                                    path_s_cost = path_cost
                            
                         
                for ij in set(link_flow_fresh_set):
                    self.link[ij].updateCost()
#            
            gap = gapFunction()
            
            tmpOut = str(self.time.time()-t1) +   "\t"    + str(gap)
            BFW_outFile.write(tmpOut + "\n")
            print("Iteration %d: gap %f" % (iteration, gap))
            if gap < targetGap:
                BFW_outFile.close()
                print(iteration)
                break
                
    def calcualtePath(self):
        a = 0
        for origin in self.origin_set:
            for OD in [OD for OD in self.ODpair if self.ODpair[OD].origin == origin]:
                 for p in self.ODpair[OD].Path_Set:
                        a += 1
        return a

    def readFromFiles(self, networkFile, demandFile):
        """
        Reads network data from a pair of files (networkFile, containing the topology,
        and demandFile, containing the OD matrix), then do some basic checks on
        the input data (validate) and build necessary data structures (finalize).
        """
        self.readNetworkFile(networkFile)
        self.readDemandFile(demandFile)
        self.validate()
        self.finalize()

    def readNetworkFile(self, networkFileName):
        """
        Reads network topology data from the TNTP data format.  In keeping with
        this format, the zones/centroids are assumed to have the lowest node
        IDs (1, 2, ..., numZones).
        """
        try:
            with open(networkFileName, "r") as networkFile:
                fileLines = networkFile.read().splitlines()

                # Set default parameters for metadata, then read
                self.numNodes = None
                self.numLinks = None
                self.numZones = None
                self.firstThroughNode = 0
                metadata = utils.readMetadata(fileLines)

                try:
                    self.numNodes = int(metadata['NUMBER OF NODES'])
                    self.numLinks = int(metadata['NUMBER OF LINKS'])
                    if self.numZones != None:
                        if self.numZones != int(metadata['NUMBER OF ZONES']):
                            print(
                                "Error: Number of zones does not match in network/demand files.")
                            raise utils.BadFileFormatException
                    else:
                        self.numZones = int(metadata['NUMBER OF ZONES'])
                    self.firstThroughNode = int(metadata['FIRST THRU NODE'])
                except KeyError:  # KeyError
                    print(
                        "Warning: Not all metadata present, error checking will be limited and code will proceed as though all nodes are through nodes.")
                self.tollFactor = float(metadata.setdefault('TOLL FACTOR', 0))
                self.distanceFactor = float(
                    metadata.setdefault('DISTANCE FACTOR', 0))

                for line in fileLines[metadata['END OF METADATA']:]:
                    # Ignore comments and blank lines
                    line = line.strip()
                    commentPos = line.find("~")
                    if commentPos >= 0:  # strip comments
                        line = line[:commentPos]

                    if len(line) == 0:
                        continue

                    data = line.split()
                    if len(data) < 11 or data[10] != ';':
                        print("Link data line not formatted properly:\n '%s'" % line)
                        raise utils.BadFileFormatException

                    # Create link
                    linkID = '(' + str(data[0]).strip() + \
                        "," + str(data[1]).strip() + ')'

                    self.link[linkID] = Link(self,
                                             # head and tail
                                             int(data[0]), int(data[1]),
                                             float(data[2]),   # capacity
                                             float(data[3]),   # length
                                             float(data[4]),   # free-flow time
                                             float(data[5]),   # BPR alpha
                                             float(data[6]),   # BPR beta
                                             float(data[7]),   # Speed limit
                                             float(data[8]),   # Toll
                                             data[9])          # Link type

                    # Create nodes if necessary
                    if data[0] not in self.node:  # tail
                        self.node[int(data[0])] = Node(
                            True if int(data[0]) <= self.numZones else False)
                    if data[1] not in self.node:  # head
                        self.node[int(data[1])] = Node(
                            True if int(data[1]) <= self.numZones else False)

        except IOError:
            print("\nError reading network file %s" % networkFile)
            traceback.print_exc(file=sys.stdout)

    def readDemandFile(self, demandFileName):
        """
        Reads demand (OD matrix) data from a file in the TNTP format.
        """
        try:
            with open(demandFileName, "r") as demandFile:
                fileLines = demandFile.read().splitlines()
                self.totalDemand = 0

                # Set default parameters for metadata, then read
                self.totalDemandCheck = None

                metadata = utils.readMetadata(fileLines)
                try:
                    self.totalDemandCheck = float(metadata['TOTAL OD FLOW'])
                    if self.numZones != None:
                        if self.numZones != int(metadata['NUMBER OF ZONES']):
                            print(
                                "Error: Number of zones does not match in network/demand files.")
                            raise utils.BadFileFormatException
                    else:
                        self.numZones = int(metadata['NUMBER OF ZONES'])

                except KeyError:  # KeyError
                    print(
                        "Warning: Not all metadata present in demand file, error checking will be limited.")

                for line in fileLines[metadata['END OF METADATA']:]:
                    # Ignore comments and blank lines
                    line = line.strip()
                    commentPos = line.find("~")
                    if commentPos >= 0:  # strip comments
                        line = line[:commentPos]
                    if len(line) == 0:
                        continue

                    data = line.split()

                    if data[0] == 'Origin':
                        origin = int(data[1])
                        self.origin_set.append(origin)
                        continue

                    # Two possibilities, either semicolons are directly after values or there is an intervening space
                    if len(data) % 3 != 0 and len(data) % 4 != 0:
                        print("Demand data line not formatted properly:\n %s" % line)
                        raise utils.BadFileFormatException

                    for i in range(int(len(data) // 3)):
                        destination = int(data[i * 3])
                        check = data[i * 3 + 1]
                        demand = data[i * 3 + 2]
                        demand = float(demand[:len(demand)-1])
                        if check != ':':
                            print(
                                "Demand data line not formatted properly:\n %s" % line)
                            raise utils.BadFileFormatException
                        ODID = str(origin) + '->' + str(destination)
                        self.ODpair[ODID] = OD(origin, destination, demand)
                        self.totalDemand += demand

        except IOError:
            print("\nError reading network file %s" % networkFile)
            traceback.print_exc(file=sys.stdout)

    def validate(self):
        """
        Perform some basic validation checking of network, link, and node
        data to ensure reasonableness and consistency.
        """
        valid = True

        # Check that link information is valid
        for ij in self.link:
            valid = valid and self.link[ij].head in self.node
            valid = valid and self.link[ij].tail in self.node
            if not valid:
                print("Error: Link tail/head not found: %s %s" %
                      (self.link[ij].tail, self.link[ij].head))
                raise utils.BadFileFormatException
            valid = valid and self.link[ij].capacity >= 0
            valid = valid and self.link[ij].length >= 0
            valid = valid and self.link[ij].freeFlowTime >= 0
            valid = valid and self.link[ij].alpha >= 0
            valid = valid and self.link[ij].beta >= 0
            valid = valid and self.link[ij].speedLimit >= 0
            valid = valid and self.link[ij].toll >= 0
            if not valid:
                print("Link %s has negative parameters." % ij)

        # Then check that all OD pairs are in range
        for ODpair in self.ODpair:
            (origin, destination) = (
                self.ODpair[ODpair].origin, self.ODpair[ODpair].destination)
            valid = valid and origin in self.node
            valid = valid and destination in self.node
            if not valid:
                print("Error: Origin/destination %s not found" % ODpair)
                raise utils.BadFileFormatException
            valid = valid and self.node[origin].isZone == True
            valid = valid and self.node[destination].isZone == True
            if not valid:
                print(
                    "Error: Origin/destination %s does not connect two zones" % str(ODpair))
                raise utils.BadFileFormatException
            valid = valid and self.ODpair[ODpair].demand >= 0
            if not valid:
                print("Error: OD pair %s has negative demand" % ODpair)
                raise utils.BadFileFormatException

        # Now error-check using metadata
        if self.numNodes != None and len(self.node) != self.numNodes:
            print("Warning: Number of nodes implied by network file %d different than metadata value %d" % (
                len(self.node), self.numNodes))
            self.numNodes = len(self.node)
        if self.numLinks != None and len(self.link) != self.numLinks:
            print("Warning: Number of links given in network file %d different than metadata value %d" % (
                len(self.link), self.numLinks))
            self.numLinks = len(self.link)
        if self.numZones != None and len([i for i in self.node if self.node[i].isZone == True]) != self.numZones:
            print("Warning: Number of zones given in network file %d different than metadata value %d" % (
                len([i for i in self.node if self.node[i].isZone == True]), self.numZones))
            self.numLinks = len(self.link)
        if self.totalDemandCheck != None:
            if self.totalDemand != self.totalDemandCheck:
                print("Warning: Total demand is %f compared to metadata value %f" % (
                    self.totalDemand, self.totalDemandCheck))

    def finalize(self):
        """
        Establish the forward and reverse star lists for nodes, initialize flows and
        costs for links and OD pairs.
        """
        # Establish forward/reverse star lists, set travel times to free-flow
        for i in self.node:
            self.node[i].forwardStar = list()
            self.node[i].reverseStar = list()

        for ij in self.link:
            self.node[self.link[ij].tail].forwardStar.append(ij)
            self.node[self.link[ij].head].reverseStar.append(ij)
            self.link[ij].cost = self.link[ij].freeFlowTime + self.link[ij].length * \
                self.distanceFactor + self.link[ij].toll * self.tollFactor
            self.link[ij].flow = 0

In [ ]:
t2=time.time()
theta = math.exp(100)
# net = Network(r"C:\Users\gonghuatian/braess_net", r"C:\Users\gonghuatian/braess_trips")
net = Network(r"SF_nett", r"SF_trips")
# net = Network(r"Anaheim_nett", r"Anaheim_trips")
# net = Network("berlin-tiergarten_nett", "berlin-tiergarten_trips")
# net = Network("berlin-mitte-center_net", "berlin-mitte-center_trips")
# net = Network(r"ChicagoSketch_net", r"ChicagoSketch_trips")
# net = Network("Anaheim_net", "Anaheim_trips")
# net = Network("berlin-mitte-center_net", "berlin-mitte-center_trips")
# net = Network("Winnipeg_net", "Winnipeg_trips")
# net = Network("braess_net1", "braess_trips1")
# net = Network("berlin-mitte-prenzlauerberg-friedrichshain-center_net", "berlin-mitte-prenzlauerberg-friedrichshain-center_trips")

net.improvedGradientProjection(maxIterations = 4000,targetGap = 1e-10,gapFunction = net.relativeGap)
t1=time.time()
print("共用时：%.6fs"%(t1-t2))
net.writeUEresults()